softmax回归

In [ ]:
import torch
import torchvision
from torch.utils import data
from torchvision import transforms
from d2l import torch as d2l
from IPython import display
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'


def get_dataloader_workers():
    return 0

def load_data_fashion_mnist(batch_size, resize=None):
    trans = [transforms.ToTensor()]
    if resize:
        trans.insert(0,transforms.Resize(resize))
    trans = transforms.Compose(trans)
    mnist_train = torchvision.datasets.FashionMNIST(root="01_data/01_DataSet_FashionMNIST",train=True,transform=trans,download=True)
    mnist_test = torchvision.datasets.FashionMNIST(root="01_data/01_DataSet_FashionMNIST",train=False,transform=trans,download=True)
    return (data.DataLoader(mnist_train, batch_size, shuffle=True, num_workers=get_dataloader_workers()),
           data.DataLoader(mnist_train, batch_size, shuffle=True, num_workers=get_dataloader_workers()))


batch_size = 256
train_iter, test_iter = load_data_fashion_mnist(batch_size)

num_inputs = 784
num_outputs = 10
w = torch.normal(0,0.01,size=(num_inputs,num_outputs),requires_grad=True)  # 用正态分布采样进行初始化
b = torch.zeros(num_outputs,requires_grad=True)  # 初始为0

# softmax函数
def softmax(X):
    X_exp = torch.exp(X)
    partition = X_exp.sum(1,keepdim=True)  # 对 X_exp 的每一行进行求和操作，并保持结果的维度和原始数据相同。这样做是为了计算每个样本的和，而不是对整个数据集进行求和。
    return X_exp / partition

# 实现 softmax回归
def net(X):
    return softmax(torch.matmul(X.reshape((-1,w.shape[1])),w)+b)  # 得分用 softmax函数处理

# 交叉熵损失函数，其中 y_hat是模型的预测结果,y是真实标签
def cross_entropy(y_hat,y):
    return -torch.log(y_hat[range(len(y_hat)),y])

# 准确率函数
def accuracy(y_hat,y):
    if len(y_hat.shape) > 1 and y_hat.shape[1] > 1:
        y_hat = y_hat.argmax(axis=1)  # 取预测中概率最高的类别
    cmp = y_hat.type(y.dtype) == y  # 创建一个布尔类型的张量 cmp，用来表示预测结果与真实标签是否相等。将 y_hat转换为与 y相同数据类型后进行比较
    return float(cmp.type(y.dtype).sum())  # 返回正确率

# 准确率评估函数
def evaluate_accuracy(net,data_iter,loss,updater):
    if isinstance(net,torch.nn.Module):
        net.eval()  # 将模型设置为评估模式，这是为了确保模型不进行梯度计算，并且不会对模型的参数进行更新
    metric = Accumulator(3)
    for X, y in train_iter:
        y_hat = net(X)
        l = loss(y_hat, y)  # 计算损失
        if isinstance(updater, torch.optim.Optimizer):  # 如果 updater是 pytorch的优化器的话
            updater.zero_grad()
            l.mean().backward()  # 这里对loss取了平均值出来
            updater.step()
            metric.add(float(l) * len(y), accuracy(y_hat, y), y.size().numel())  # 总的训练损失、样本正确数、样本总数
        else:  # 如果 updater不是 pytorch的优化器，表示不需要更新模型的参数，那么代码会对损失l求和，并通过调用updater函数来更新其他参数
            l.sum().backward()
            updater(X.shape[0])
            metric.add(float(l.sum()), accuracy(y_hat, y), y.numel())
    return metric[0] / metric[2], metric[1] / metric[2]


class Accumulator:
    def __init__(self, n):
        self.data = [0, 0] * n

    def add(self, *args):
        self.data = [a + float(b) for a, b in zip(self.data, args)]  # zip函数把两个列表第一个位置元素打包、第二个位置元素打包....

    def reset(self):
        self.data = [0.0] * len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

# 训练函数
def train_epoch_ch3(net, train_iter, loss, updater):
    if isinstance(net, torch.nn.Module):
        net.train() # 开启训练模式
    metric = Accumulator(3)
    for X, y in train_iter:
        y_hat = net(X)
        l = loss(y_hat,y) # 计算损失
        if isinstance(updater, torch.optim.Optimizer): # 如果updater是pytorch的优化器的话
            updater.zero_grad()
            l.mean().backward()  # 这里对loss取了平均值出来
            updater.step()
            metric.add(float(l)*len(y),accuracy(y_hat,y),y.size().numel()) # 总的训练损失、样本正确数、样本总数
        else:
            l.sum().backward()
            updater(X.shape[0])
            metric.add(float(l.sum()),accuracy(y_hat,y),y.numel())
    return metric[0] / metric[2], metric[1] / metric[2]

# 动画绘制
class Animator:
    def __int__(self,x_label=None,y_label=None,legend=None,
                x_lim=None,y_lim=None,x_scale='linear',y_scale='linear',
                fmts=('-','m--','g-','r:'),n_rols=1,n_cols=1,figsize=(3.5,2.5)):
        if legend is None:
            legend = []
        d2l.use_svg_display()
        self.fig, self.axis = d2l.plt.subplots(n_rols,n_cols,figsize=figsize)
        if n_rols * n_cols == 1:
            self.axis = [self.axes,]
        self.config_axes = lambda: d2l.set_axes(self.axes[0],x_label,y_label,x_lim,y_lim,x_scale,y_scale,legend)
        self.X, self.Y, self.fmts = None, None, fmts

    def add(self, x, y):
        if not hasattr(y, "__len__"):
            y = [y]
        n = len(y)
        if not hasattr(x, "__len__"):
            x = [x] * n
        if not self.X:
            self.X = [[] for _ in range(n)]
        if not self.Y:
            self.Y = [[] for _ in range(n)]
        for i, (a,b) in enumerate(zip(x,y)):
            if a is not None and b is not None:
                self.X[i].append(a)
                self.Y[i].append(b)
        self.axes[0].cla()
        for x, y, fmt in zip(self.X, self.Y, self.fmts):
            self.axes[0].plot(x, y, fmt)
        self.config_axes()
        display.display(self.fig)
        display.clear_output(wait=True)

# 总训练函数
def train_ch3(net,train_iter,test_iter,loss,num_epochs,updater):
    animator = Animator(xlabel='epoch',xlim=[1,num_epochs],ylim=[0.3,0.9],
                       legend=['train loss','train acc','test acc'])
    for epoch in range(num_epochs):  # 变量num_epochs遍数据
        train_metrics = train_epoch_ch3(net,train_iter,loss,updater) # 返回两个值，一个总损失、一个总正确率
        test_acc = evaluate_accuracy(net, test_iter) # 测试数据集上评估精度，仅返回一个值，总正确率
        animator.add(epoch+1,train_metrics+(test_acc,)) # train_metrics+(test_acc,) 仅将两个值的正确率相加，
    train_loss, train_acc = train_metrics

# 小批量随即梯度下降来优化模型的损失函数
lr = 0.1
def updater(batch_size):
    return d2l.sgd([w,b],lr,batch_size)

num_epochs = 10
train_ch3(net,train_iter,test_iter,cross_entropy,num_epochs,updater)

softmax回归（使用框架）

In [ ]:
import torch
from torch import nn
from d2l import torch as d2l
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

batch_size = 256
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size)

# Softmax回归的输出是一个全连接层，PyTorch不会隐式地调整输入的形状
# 因此，定义展平层(flatten)在线性层前调整网络输入的形状
net = nn.Sequential(nn.Flatten(),nn.Linear(784,10))

# 初始化权重的函数
def init_weights(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight,std=0.01)  # 使用正态分布来初始化该层的权重，均值为 0，方差为 0.01

net.apply(init_weights)
print(net.apply(init_weights))

# 交叉熵损失函数
loss = nn.CrossEntropyLoss()
# 使用学习率为0.1的小批量随即梯度下降作为优化算法
trainer = torch.optim.SGD(net.parameters(),lr=0.1)

num_epochs = 10
d2l.train_ch3(net,train_iter,test_iter,loss,num_epochs,trainer)
# 绘图
d2l.plt.show()